# Interactive web mapping
In this notebook I use the [ipyleaflet]() package to show how it can be used to add some interaction to a Juptyter notebook. One example is how to draw a bounding box on a map and use the geometry to filter the administrative boundary it belongs to. 

## Rectangle on a satellite image

In [1]:
import geopandas as gpd
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles, Rectangle, DrawControl
from ipywidgets import Layout

m = Map(
    basemap=basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2017-04-08"),
    center=(41.890, 12.492),
    zoom=9
)
m.add(Marker(location=(41.890, 12.492)))
m.add(Marker(location=(42.0, 13.0)))
m.add(Marker(location=(41.7, 12.6)))
rectangle = Rectangle(bounds=((42.0, 13.0), (41.7, 12.6)), transform=True)
m.add(rectangle)
m

Map(center=[41.89, 12.492], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [2]:
rectangle.bounds

[(42.0, 13.0), (41.7, 12.6)]

## Drawing a rectangle on a map

In [3]:
watercolor = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)

m = Map(layers=(watercolor, ), center=(50, 354), zoom=5)

draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

m.add(draw_control)

m

Map(center=[50, 354], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [6]:
num_features = len(feature_collection['features'])
num_features

1

In [7]:
if (num_features > 0): 
    print(feature_collection['features'][num_features - 1]['geometry']['coordinates'])

[[[360.775946, 47.243441], [360.775946, 49.405255], [365.435869, 49.405255], [365.435869, 47.243441], [360.775946, 47.243441]]]


## Getting the bounding box of an area of interest
Often we need to select an area of interestt using its bounding box. For example we want to select an area that contains certain features, e.g. buildings with some attributes of interest. The bounding box can be used to select satellite images that cover that area.

In [12]:
milan_gdf = countries = gpd.read_file('data/vector/italy/osm/milan_20240310.geojson')

In [13]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import json

m = Map(center=(45.465, 9.188), zoom = 11, basemap= basemaps.OpenStreetMap.Mapnik)

geo_data = GeoData(geo_dataframe = milan_gdf,
                   style={'color': 'orange', 'fillColor': 'red', 'opacity':1.0, 'weight':1.0, 'dashArray':'2', 'fillOpacity':1.0},
                   hover_style={'fillColor': 'blue' , 'fillOpacity': 1.0},
                   name = 'Milan')

m.add(geo_data)
draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}

feature_collection = {
    'type': 'FeatureCollection',
    'features': []
}

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""    
    feature_collection['features'].append(geo_json)

draw_control.on_draw(handle_draw)

m.add(draw_control)
m.add(LayersControl())
m

Map(center=[45.465, 9.188], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [10]:
num_features = len(feature_collection['features'])
num_features

1

In [11]:
if (num_features > 0): 
    print(feature_collection['features'][num_features - 1]['geometry']['coordinates'])

[[[9.163687, 45.493954], [9.163687, 45.523788], [9.218639, 45.523788], [9.218639, 45.493954], [9.163687, 45.493954]]]
